In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/reviews.csv")

In [5]:
display(df.head())

display(df.describe())

,listing_id,date
0,109,2011-08-15
1,109,2016-05-15
2,2708,2014-06-09
3,2708,2014-06-23
4,2708,2015-07-25


,listing_id
count,1.075393e+06
mean,1.639552e+07
std,1.145206e+07
min,1.090000e+02
25%,6.364732e+06
50%,1.554980e+07
75%,2.392072e+07
max,4.578991e+07
